In [23]:
import pandas as pd 
from tabulate import tabulate
import re
import os

In [2]:
df = pd.read_excel('/home/vishwan/Projects/PAN Card Validation in PYTHON - Scripts/PAN Number Validation Dataset.xlsx')

In [3]:
print(df.head(10))

    Pan_Numbers
0    VGLOD3180G
1    PHOXD7232L
2    MGEPH6532A
3    JJCHK4574O
4    XTQIJ2330L
5    HTJYM3835H
6    YQTAP6661X
7    hvofe5635y
8  hyuij7902r  
9    idsmt3429e


In [4]:
print('Total records = ',len(df))
total_records = len(df)

Total records =  10000


# Data Cleaning

In [5]:
df["Pan_Numbers"] = df['Pan_Numbers'].astype('string').str.strip().str.upper()
print(df.head(10))

  Pan_Numbers
0  VGLOD3180G
1  PHOXD7232L
2  MGEPH6532A
3  JJCHK4574O
4  XTQIJ2330L
5  HTJYM3835H
6  YQTAP6661X
7  HVOFE5635Y
8  HYUIJ7902R
9  IDSMT3429E


In [6]:
print(df[df['Pan_Numbers']==''])

     Pan_Numbers
5019            
5020            


In [7]:
# Displaying Null Values
print(df[df['Pan_Numbers'].isna()])

     Pan_Numbers
5022        <NA>
5027        <NA>
5033        <NA>
5043        <NA>
5057        <NA>
...          ...
9961        <NA>
9972        <NA>
9986        <NA>
9987        <NA>
9997        <NA>

[965 rows x 1 columns]


In [8]:
# Removing Missing and NA records
df = df.replace({"Pan_Numbers":''}, pd.NA).dropna(subset="Pan_Numbers")

In [9]:
print(df.head(10))

  Pan_Numbers
0  VGLOD3180G
1  PHOXD7232L
2  MGEPH6532A
3  JJCHK4574O
4  XTQIJ2330L
5  HTJYM3835H
6  YQTAP6661X
7  HVOFE5635Y
8  HYUIJ7902R
9  IDSMT3429E


In [10]:
print('Total records = ',len(df))

Total records =  9033


In [11]:
print('Unique values = ', df["Pan_Numbers"].nunique())

Unique values =  9025


In [12]:
# Removing Duplicates
df = df.drop_duplicates(subset="Pan_Numbers", keep='first')
print('Total records = ',len(df))

Total records =  9025


# Validation

In [13]:
def has_adjacent_repitition(pan):
    return any(pan[i] == pan[i+1] for i in range(len(pan)-1))

In [14]:
def is_sequencial(pan): #ABCDE , ACFGT
    return all(ord(pan[i+1]) - ord(pan[i]) == 1 for i in range(len(pan)-1))

In [15]:
def is_valid_pan(pan):
    if len(pan) != 10:
        return False 
    
    if not re.match(r'^[A-Z]{5}[0-9]{4}[A-Z]$', pan):
        return False 
    
    if has_adjacent_repitition(pan):
        return False 
    
    if is_sequencial(pan):
        return False
    
    return True

In [16]:
df["Status"] = df["Pan_Numbers"].apply(lambda x: "Valid" if is_valid_pan(x) else "Invalid")
print(df.head(10))

  Pan_Numbers   Status
0  VGLOD3180G    Valid
1  PHOXD7232L    Valid
2  MGEPH6532A    Valid
3  JJCHK4574O  Invalid
4  XTQIJ2330L  Invalid
5  HTJYM3835H    Valid
6  YQTAP6661X  Invalid
7  HVOFE5635Y    Valid
8  HYUIJ7902R    Valid
9  IDSMT3429E    Valid


In [17]:
valid_cnt = (df["Status"]=='Valid').sum()
invalid_cnt = (df["Status"]=='Invalid').sum()
missing_cnt = total_records - (valid_cnt+invalid_cnt)

print('Total records = ', total_records)
print('Valid = ', valid_cnt)
print('Invalid = ', invalid_cnt)
print('Missing = ', missing_cnt)

Total records =  10000
Valid =  3193
Invalid =  5832
Missing =  975


In [22]:
df_summary = pd.DataFrame({ "TOTAL PROCESSED RECORDS":[total_records]
                           ,"TOTAL VALID COUNT": [valid_cnt]
                           ,"TOTAL INVALID COUNT": [invalid_cnt]
                           ,"TOTAL MISSING PANS": [missing_cnt]})
print(tabulate(df_summary, headers="keys", tablefmt="psql", showindex=False))

+---------------------------+---------------------+-----------------------+----------------------+
|   TOTAL PROCESSED RECORDS |   TOTAL VALID COUNT |   TOTAL INVALID COUNT |   TOTAL MISSING PANS |
|---------------------------+---------------------+-----------------------+----------------------|
|                     10000 |                3193 |                  5832 |                  975 |
+---------------------------+---------------------+-----------------------+----------------------+


In [24]:
# Writing Data in Excel
excel_file = "PAN_VALIDATION_RESULT.xlsx"
with pd.ExcelWriter(excel_file) as writer:
    df.to_excel(writer, sheet_name="PAN Validations", index=False)
    df_summary.to_excel(writer, sheet_name="SUMMARY", index=False)

if os.path.exists(excel_file):
    print("File created successfully!")
else:
    print("File creation failed.")

File created successfully!
